In [39]:
import pandas as pd
import cv2 


df = pd.read_csv("/Users/menghang/Desktop/ml-dev/football-server/YOLOX_outputs/yolox_m_mix_det/track_vis/df2.txt")

In [40]:
ball = df[df["label"] == "ball"]

In [41]:
ball_possession = []

In [42]:
import numpy as np

def find_nearest_coordinate(point, coordinates):
    distances = np.linalg.norm(coordinates - point, axis=1)  # Calculate Euclidean distances
    nearest_index = np.argmin(distances)  # Find index of the minimum distance
    nearest_coordinate = coordinates[nearest_index]  # Get the nearest coordinate
    return nearest_coordinate, nearest_index

coordinates = np.array([[2, 4], [1, 3], [5, 6], [0, 1]])
point = np.array([3, 5])

nearest_coordinate , nearest_index = find_nearest_coordinate(point, coordinates)
print("Nearest Coordinate:", nearest_coordinate, nearest_index)

Nearest Coordinate: [2 4] 0


In [43]:
for i, row in ball.iterrows():
    frame = df[df["frame_id"] == row["frame_id"]]

    ball_center = [row["x_homo"], row["y_homo"]]   
    players_centers = []
    for j, f in frame.iterrows(): 
        if(f["label"] != "ball"):
         players_centers.append([f["x_homo"], f["y_homo"]])
    
    player, index = find_nearest_coordinate(np.array(ball_center), np.array(players_centers))
    b = frame.iloc[index]
    while b.color == "red" or b.color == "green":
        index += 1
        b = frame.iloc[index]
    
    print(b.color)
    
    if i == 30:
       break
    ball_possession.append(b["color"])
                          
                          
                                        

blue
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
blue
white
white
blue
white
white
white
white
white
white
white
blue
white
white
blue
white
blue
white
blue
white
white
white
white
white
white
blue
white
white
white
white
blue
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
blue
white
white
white
white
white
white
white
white
blue
white
white
white
white
white
white
white
white
white
white
white
white
white
white
blue
blue
white
white
white
white
white
white
white
white
white
white
white
blue
blue
white
white
white
blue
white
white
blue
white
blue
white
blue
blue
blue
white
blue
blue
blue
white
blue
blue
blue
white
blue
white
white
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
white
blue
blue
blue
blue
blue
white
blue
blue
blue
wh

In [44]:
occrence = {"white" : 0, "blue":0, "red": 0, "green": 0}
for i in ball_possession:
    occrence[i] += 1


In [45]:
occrence

{'white': 647, 'blue': 394, 'red': 0, 'green': 0}

In [46]:
import matplotlib
import PIL

from PIL import Image, ImageFont, ImageDraw
def possession_bar( frame: PIL.Image.Image, origin: tuple, home: float, away:float) -> PIL.Image.Image:
      
        bar_x = origin[0]
        bar_y = origin[1]
        bar_height = 29
        bar_width = 310

        ratio = home
        if ratio <= 0.25:
            ratio = 0.25

        if ratio >= 0.75:
            ratio = 0.75

        left_rectangle = (
            origin,
            [int(bar_x + ratio * bar_width), int(bar_y + bar_height)],
        )

        right_rectangle = (
            [int(bar_x + ratio * bar_width), bar_y],
            [int(bar_x + bar_width), int(bar_y + bar_height)],
        )

        left_color =  (70,115,207)
        right_color = (197,197,197)

        frame = draw_counter_rectangle(
            frame=frame,
            ratio=ratio,
            left_rectangle=left_rectangle,
            left_color=left_color,
            right_rectangle=right_rectangle,
            right_color=right_color,
        )

        # Draw home text
        if True:
            home_text = (
                f"{home}"
            )

            frame = text_in_middle_rectangle(
                img=frame,
                origin=left_rectangle[0],
                width=left_rectangle[1][0] - left_rectangle[0][0],
                height=left_rectangle[1][1] - left_rectangle[0][1],
                text=home_text,
                color=(255,255,255)
            )

        # Draw away text
        if True:
            away_text = (
                f"{away}"
            )

            frame = text_in_middle_rectangle(
                img=frame,
                origin=right_rectangle[0],
                width=right_rectangle[1][0] - right_rectangle[0][0],
                height=right_rectangle[1][1] - right_rectangle[0][1],
                text=away_text,
                color=(255,255,255)
            )

        return frame

def text_in_middle_rectangle(
        img: PIL.Image.Image,
        origin: tuple,
        width: int,
        height: int,
        text: str,
        font: PIL.ImageFont = None,
        color=(255, 255, 255),
    ) -> PIL.Image.Image:
        

        draw = PIL.ImageDraw.Draw(img)

        if font is None:
            font = PIL.ImageFont.truetype("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/fonts/Gidole-Regular.ttf", size=24)

        w, h = draw.textsize(text, font=font)
        text_origin = (
            origin[0] + width / 2 - w / 2,
            origin[1] + height / 2 - h / 2,
        )

        draw.text(text_origin, text, font=font, fill=color)

        return img
def draw_counter_rectangle(
        frame: PIL.Image.Image,
        ratio: float,
        left_rectangle: tuple,
        left_color: tuple,
        right_rectangle: tuple,
        right_color: tuple,
    ) -> PIL.Image.Image:
      


        if ratio <= 0.15:
            left_rectangle[1][0] += 20

            frame = half_rounded_rectangle(
                frame,
                rectangle=left_rectangle,
                color=left_color,
                radius=15,
            )

            frame = half_rounded_rectangle(
                frame,
                rectangle=right_rectangle,
                color=right_color,
                left=True,
                radius=15,
            )
        else:
            right_rectangle[0][0] -= 10

            frame = half_rounded_rectangle(
                frame,
                rectangle=right_rectangle,
                color=right_color,
                left=True,
                radius=15,
            )

            frame = half_rounded_rectangle(
                frame,
                rectangle=left_rectangle,
                color=left_color,
                radius=15,
            )

        return frame
def half_rounded_rectangle(
        img: PIL.Image.Image,
        rectangle: tuple,
        color: tuple,
        radius: int = 15,
        left: bool = False,
    ) -> PIL.Image.Image:
       
        overlay = img.copy()
        draw = PIL.ImageDraw.Draw(overlay, "RGBA")
        draw.rounded_rectangle(rectangle, radius, fill=color)

        height = rectangle[1][1] - rectangle[0][1]
        stop_width = 13

        if left:
            draw.rectangle(
                (
                    rectangle[0][0] + 0,
                    rectangle[1][1] - height,
                    rectangle[0][0] + stop_width,
                    rectangle[1][1],
                ),
                fill=color,
            )
        else:
            draw.rectangle(
                (
                    rectangle[1][0] - stop_width,
                    rectangle[1][1] - height,
                    rectangle[1][0],
                    rectangle[1][1],
                ),
                fill=color,
            )
        return overlay


In [ ]:
from PIL import Image, ImageFont

img = Image.open("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/images/possession_board.png")

possession_bar(img, (30,200), 0.85,0.15)

In [47]:
cap = cv2.VideoCapture("/Users/menghang/Desktop/ml-dev/football-server/video_demo/concat.avi")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) ) # float
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)


out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 25, (800,600))

ball_frame = list(ball["frame_id"])

frame_count = 0


ball_found = 0

possession = {"white": 0, "blue": 0 }

possesion_board =  Image.open("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/images/possession_board.png")


while True:     
    success, frame = cap.read()
    if success:
        if frame_count in ball_frame: 
            ball_found += 1
            possession[ball_possession[ball_found-1] ] += 1 
            home = round(possession["blue"] / ball_found, 2)    
            board = np.array( possession_bar(possesion_board, (30,200), home , round(1 - home,2)))
            board = cv2.cvtColor(board,  cv2.COLOR_RGB2BGR)
            board = cv2.resize(board, (800,600))
            out.write(board)

        
        if ball_found == 0:
            board = np.array( possession_bar(possesion_board, (30,200), 0.5,0.5))
            board = cv2.cvtColor(board,  cv2.COLOR_RGB2BGR)
            board = cv2.resize(board, (800,600))
           
            out.write(board)
        else:
            out.write(board)
        frame_count += 1
    
  
    
    else:
        break
out.release()
cap.release()
cv2.destroyAllWindows()


In [ ]:
from PIL import Image, ImageFont

img = Image.open("/Users/menghang/Desktop/ml-dev/football-server/forked_repo/soccer-video-analytics/images/possession_board.png")

possession_bar(img, (30,200), 0.95,0.05)

In [48]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 

detection = cv2.VideoCapture("demo_video/concat.avi")
possession = cv2.VideoCapture("demo_video/ball_possession.mp4")
template = cv2.VideoCapture("demo_video/template_2d.avi")
blue = cv2.VideoCapture("demo_video/blue.mp4")
white = cv2.VideoCapture("demo_video/white.mp4")


writer = cv2.VideoWriter("video.mp4", cv2.VideoWriter_fourcc(*'MJPG'), 25, (1060, 2160) )




while True:
    d, det = detection.read()
    p, po = possession.read()
    t, tp = template.read()
    b, bl = blue.read()
    w, wh = white.read()

    if d and p and t and b and w:

        bl = bl[536:536+848,46: 46 + 1229]
        wh = wh[536:536+848,46: 46 + 1229]

        heatmap = np.hstack([bl, wh])
        shape = (1060, 540)
        det = cv2.resize(det, shape, interpolation=cv2.INTER_AREA)
        po =  cv2.resize(po, shape, interpolation=cv2.INTER_AREA)
        tp =  cv2.resize(tp, shape, interpolation=cv2.INTER_AREA)
        heatmap =  cv2.resize(heatmap, shape, interpolation=cv2.INTER_AREA)
        
        vstck = np.vstack([det, tp, po , heatmap])

        cv2.imshow("frame", vstck)
        # key = cv2.waitKey(0)
        # if key == ord('q'):
        #     cv2.destroyAllWindows()
        #     cv2.waitKey(1)
        #     break
        writer.write(vstck)
    else:
        break
    

writer.release()




: 

In [38]:
import pandas as pd
l = pd.read_csv("YOLOX_outputs/yolox_m_mix_det/track_vis/left.txt",usecols=["frame_id","id","x","y","w","h","score","label","x_center","y_center","color","x_homo","y_homo"])
m = pd.read_csv("YOLOX_outputs/yolox_m_mix_det/track_vis/mid.txt",usecols=["frame_id","id","x","y","w","h","score","label","x_center","y_center","color","x_homo","y_homo"])
r = pd.read_csv("YOLOX_outputs/yolox_m_mix_det/track_vis/right.txt",usecols=["frame_id","id","x","y","w","h","score","label","x_center","y_center","color","x_homo","y_homo"])

df = pd.concat([l,m,r])
df = df.sort_values(by=['frame_id'], ascending=[True])

df.to_csv("YOLOX_outputs/yolox_m_mix_det/track_vis/df2.txt")